In [ ]:
# 10-1
!mkdir chap10
%cd ./chap10

In [2]:
# 10-3
import random
import glob
from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel

# BERTの日本語モデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 10-4
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 

--2025-11-27 13:52:59--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M  8.19MB/s 時間 1.0s       

2025-11-27 13:53:01 (8.19 MB/s) - `ldcc-20140209.tar.gz' へ保存完了 [8855190/8855190]



In [5]:
# 10-5（Mac対応：MPS/CPU/CUDA 自動切替、説明コメント付き）

import glob
import numpy as np
import torch
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertModel

# --- モデル名（未定義なら既定を設定） ---
try:
    MODEL_NAME
except NameError:
    MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

# --- デバイス自動選択：MPS → CUDA → CPU の順で利用 ---
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")  # Apple Silicon (Metal Performance Shaders)
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA CUDA
else:
    device = torch.device("cpu")  # CPU フォールバック

print(f"# device = {device}")

# ================================================================
# 目的
# - Livedoorニュース9カテゴリから本文を取り出し、
#   BERT の最終層隠れ状態を [PAD] を無視した平均でプーリングして文ベクトル化。
#
# 理論メモ
# - [CLS] ではなく masked mean pooling を使う理由：
#   事前学習BERTの [CLS] は次文予測などタスク特化バイアスが強く、
#   文章意味の幾何（コサイン距離等）では平均プーリングの方が一貫しやすい知見がある。
#   ただし vanilla BERT は文埋め込み専用に訓練されていないため、
#   Sentence-BERT系などの専用モデルや微調整でさらに精度が上がる。
# ================================================================

# カテゴリーのリスト
category_list = [
    "dokujo-tsushin",
    "it-life-hack",
    "kaden-channel",
    "livedoor-homme",
    "movie-enter",
    "peachy",
    "smax",
    "sports-watch",
    "topic-news",
]

# トークナイザとモデルのロード（推論モードへ）
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME).to(device).eval()

max_length = 256
sentence_vectors = []  # 各記事の文ベクトル（形状: [H]）
labels = []  # カテゴリID

for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f"./text/{category}/{category}*"):
        # --- 本文抽出：先頭3行をメタとみなし、4行目以降を本文として結合 ---
        # 文字コードが不明な場合は encoding 指定を調整（例: encoding='utf-8'）
        with open(file, "r") as f:
            lines = f.read().splitlines()
        text = "\n".join(lines[3:])

        # --- 符号化：固定長化（max_length）＋PAD付与、CLS/SEPは自動付与 ---
        encoding = tokenizer(
            text,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        # 張り付け先デバイスへ移動（MPS/CUDA/CPU）
        encoding = {k: v.to(device) for k, v in encoding.items()}
        attention_mask = encoding["attention_mask"]  # [1, T]（実トークン=1, PAD=0）

        # --- 文ベクトル化：masked mean pooling（[PAD]無視平均）---
        # last_hidden_state: [B=1, T, H]
        with torch.no_grad():
            output = model(**encoding)
            last_hidden_state = output.last_hidden_state
            mask = attention_mask.unsqueeze(-1)  # [1, T, 1]
            summed = (last_hidden_state * mask).sum(dim=1)  # [1, H]
            count = mask.sum(dim=1).clamp(min=1)  # [1, 1]（ゼロ割防止）
            pooled = summed / count  # [1, H]

        sentence_vectors.append(pooled[0].detach().cpu().numpy())
        labels.append(label)

# --- numpy 配列へ整形 ---
sentence_vectors = np.vstack(sentence_vectors)  # [N_doc, H]
labels = np.array(labels)  # [N_doc]

# 以降：可視化/分類器学習などに sentence_vectors, labels を使用
# 例）sklearn のロジスティック回帰やt-SNE/UMAP可視化など

# device = mps


100%|██████████| 9/9 [42:40<00:00, 284.49s/it]   
